In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Conv1D, MaxPooling1D, Dense, Dropout, BatchNormalization, 
    GlobalAveragePooling1D, Input, Activation, SpatialDropout1D, 
    LSTM, Bidirectional, Multiply, Reshape, LeakyReLU
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K
import tensorflow as tf
import random

# ===================== DATA AUGMENTATION =====================
def augment_signal(segment, augmentation_prob=0.5):
    """Apply random augmentation to EEG segment"""
    if np.random.random() > augmentation_prob:
        return segment
    
    aug_type = np.random.choice(['noise', 'scale', 'shift', 'time_shift'], p=[0.3, 0.3, 0.2, 0.2])
    
    if aug_type == 'noise':
        noise_level = np.random.uniform(0.01, 0.05)
        noise = np.random.normal(0, noise_level, segment.shape)
        return segment + noise
    elif aug_type == 'scale':
        scale = np.random.uniform(0.9, 1.1)
        return segment * scale
    elif aug_type == 'shift':
        shift = np.random.uniform(-0.1, 0.1)
        return segment + shift
    elif aug_type == 'time_shift':
        shift_amount = np.random.randint(-20, 20)
        return np.roll(segment, shift_amount)
    
    return segment


def augment_batch(X_batch, y_batch, augmentation_prob=0.5):
    """Apply augmentation to a batch of data"""
    X_augmented = np.zeros_like(X_batch)
    
    for i in range(len(X_batch)):
        if y_batch[i] == 1:
            prob = augmentation_prob * 1.5
        else:
            prob = augmentation_prob
        
        X_augmented[i, :, 0] = augment_signal(X_batch[i, :, 0], prob)
    
    return X_augmented


class AugmentedDataGenerator(tf.keras.utils.Sequence):
    """Custom data generator with real-time augmentation"""
    def __init__(self, X, y, batch_size=32, augmentation_prob=0.5, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.augmentation_prob = augmentation_prob
        self.shuffle = shuffle
        self.indices = np.arange(len(self.X))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))
    
    def __getitem__(self, index):
        start_idx = index * self.batch_size
        end_idx = min((index + 1) * self.batch_size, len(self.X))
        batch_indices = self.indices[start_idx:end_idx]
        
        X_batch = self.X[batch_indices].copy()
        y_batch = self.y[batch_indices]
        
        X_batch = augment_batch(X_batch, y_batch, self.augmentation_prob)
        
        return X_batch, y_batch
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


# ===================== SE BLOCK =====================
class SEBlock(tf.keras.layers.Layer):
    """Squeeze-and-Excitation Block for channel attention"""
    def __init__(self, reduction=8, **kwargs):
        super(SEBlock, self).__init__(**kwargs)
        self.reduction = reduction
    
    def build(self, input_shape):
        channels = input_shape[-1]
        self.squeeze = GlobalAveragePooling1D()
        self.fc1 = Dense(channels // self.reduction, activation='relu', kernel_initializer='he_normal')
        self.fc2 = Dense(channels, activation='sigmoid', kernel_initializer='he_normal')
        super(SEBlock, self).build(input_shape)
    
    def call(self, inputs):
        squeeze = self.squeeze(inputs)
        excitation = self.fc1(squeeze)
        excitation = self.fc2(excitation)
        excitation = tf.reshape(excitation, [-1, 1, tf.shape(inputs)[-1]])
        return Multiply()([inputs, excitation])
    
    def get_config(self):
        config = super(SEBlock, self).get_config()
        config.update({"reduction": self.reduction})
        return config


# ===================== LOAD DATA =====================
X = np.load(r"preprocessed\ALL_X.npy")
y = np.load(r"preprocessed\ALL_y.npy")

y_encoded = np.where(y == 'NORMAL', 0, 1)

print("Original classes:", np.unique(y))
print("Binary classes: 0 (NORMAL) vs 1 (INTERICTAL + ICTAL)")
print("Binary labels distribution:", np.unique(y_encoded, return_counts=True))

X = X.reshape((X.shape[0], X.shape[1], 1))
print("Dataset shape:", X.shape)

class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print(f"Class weights: {class_weight_dict}")

os.makedirs("results", exist_ok=True)
os.makedirs("results/ensemble", exist_ok=True)


# ===================== LOSS FUNCTIONS =====================
def hybrid_focal_loss(alpha=0.75, gamma=1.7, focal_weight=0.55):
    """Hybrid: Focal + BCE"""
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        
        bce = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        focal_term = K.pow(1 - p_t, gamma)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        focal = alpha_t * focal_term * bce
        
        combined = focal_weight * focal + (1 - focal_weight) * bce
        return K.mean(combined)
    
    return loss


# ===================== MODEL BUILDING =====================
def build_model(input_shape, seed=None):
    """Build model with optional random seed for reproducibility"""
    if seed is not None:
        tf.random.set_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
    
    inputs = Input(shape=input_shape)
    
    # Block 1: Local patterns
    x = Conv1D(48, kernel_size=7, padding='same', kernel_regularizer=l2(0.002))(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = SEBlock(reduction=8)(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = SpatialDropout1D(0.3)(x)
    
    # Block 2: Mid-level features
    x = Conv1D(96, kernel_size=5, padding='same', kernel_regularizer=l2(0.002))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = SEBlock(reduction=8)(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = SpatialDropout1D(0.35)(x)
    
    # Block 3: High-level features
    x = Conv1D(128, kernel_size=3, padding='same', kernel_regularizer=l2(0.002))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = SEBlock(reduction=8)(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = SpatialDropout1D(0.4)(x)
    
    # Block 4: Deep features
    x = Conv1D(160, kernel_size=3, padding='same', kernel_regularizer=l2(0.002))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = SEBlock(reduction=8)(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = SpatialDropout1D(0.4)(x)
    
    # LSTM Temporal Modeling
    x = LSTM(64, return_sequences=False, kernel_regularizer=l2(0.001))(x)
    x = Dropout(0.4)(x)
    
    # Dense Classification
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.003))(x)
    x = Dropout(0.5)(x)
    
    x = Dense(32, activation='relu', kernel_regularizer=l2(0.003))(x)
    x = Dropout(0.5)(x)
    
    outputs = Dense(1, activation='sigmoid')(x)
    
    return Model(inputs=inputs, outputs=outputs)


# ===================== ENSEMBLE CONFIGURATION =====================
N_ENSEMBLE_MODELS = 5
ENSEMBLE_SEEDS = [42, 123, 456, 789, 1024]  # Different seeds for diversity

print("\n" + "="*80)
print(f" ENSEMBLE TRAINING: {N_ENSEMBLE_MODELS} MODELS")
print("="*80)


# ===================== CROSS-VALIDATION WITH ENSEMBLE =====================
random_state = np.random.randint(0, 10000)
print(f"\n🎲 Base random state for fold splitting: {random_state}")

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
fold_indices = [(train_val_idx, test_idx) for train_val_idx, test_idx in kfold.split(X, y_encoded)]
np.save("results/ensemble/fold_indices.npy", np.array(fold_indices, dtype=object), allow_pickle=True)

# Metrics tracking
ensemble_metrics = {
    'fold_no': [],
    'individual_accs': [],  # List of accuracies from each model
    'ensemble_acc': [],
    'ensemble_auc': [],
    'individual_aucs': [],
    'improvement': []
}

class_names = ['NORMAL', 'ALL (INTERICTAL+ICTAL)']
all_ensemble_predictions = []

for fold_no, (train_val_idx, test_idx) in enumerate(fold_indices, start=1):
    
    print(f"\n{'='*80}")
    print(f" FOLD {fold_no} - TRAINING {N_ENSEMBLE_MODELS} MODELS")
    print(f"{'='*80}\n")

    # Split data
    X_train_val, X_test = X[train_val_idx], X[test_idx]
    y_train_val, y_test = y_encoded[train_val_idx], y_encoded[test_idx]

    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.15, stratify=y_train_val, random_state=42
    )

    print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
    
    # Store predictions from all models
    fold_predictions = []
    fold_individual_accs = []
    fold_individual_aucs = []
    
    # Train ensemble models
    for model_idx in range(N_ENSEMBLE_MODELS):
        seed = ENSEMBLE_SEEDS[model_idx]
        print(f"\n{'─'*60}")
        print(f"🔸 Training Model {model_idx + 1}/{N_ENSEMBLE_MODELS} (seed={seed})")
        print(f"{'─'*60}")
        
        # Build model with specific seed
        model = build_model(input_shape=(X.shape[1], 1), seed=seed)
        
        # Compile
        model.compile(
            optimizer=Adam(learning_rate=5e-5),
            loss=hybrid_focal_loss(alpha=0.70, gamma=1.5, focal_weight=0.5),
            metrics=['accuracy']
        )
        
        # Callbacks
        early_stop = EarlyStopping(
            monitor='val_loss', 
            patience=25, 
            restore_best_weights=True, 
            verbose=0, 
            mode='min'
        )
        
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.6,
            patience=8, 
            min_lr=1e-7,
            verbose=0,
            mode='min'
        )
        
        checkpoint = ModelCheckpoint(
            f"results/ensemble/fold{fold_no}_model{model_idx+1}.weights.h5", 
            monitor='val_accuracy', 
            save_best_only=True, 
            save_weights_only=True,
            verbose=0,
            mode='max'
        )
        
        callbacks = [early_stop, reduce_lr, checkpoint]
        
        # Train with data augmentation
        train_generator = AugmentedDataGenerator(
            X_train, y_train, 
            batch_size=12, 
            augmentation_prob=0.55,
            shuffle=True
        )
        
        history = model.fit(
            train_generator,
            epochs=150,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            class_weight=class_weight_dict,
            verbose=0
        )
        
        # Load best weights
        model.load_weights(f"results/ensemble/fold{fold_no}_model{model_idx+1}.weights.h5")
        
        # Evaluate individual model
        test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
        y_pred_prob = model.predict(X_test, verbose=0)
        test_auc = roc_auc_score(y_test, y_pred_prob)
        
        fold_predictions.append(y_pred_prob)
        fold_individual_accs.append(test_acc)
        fold_individual_aucs.append(test_auc)
        
        print(f"  ✓ Model {model_idx + 1} - Acc: {test_acc:.4f}, AUC: {test_auc:.4f}")
    
    # Ensemble predictions (average)
    ensemble_pred_prob = np.mean(fold_predictions, axis=0)
    ensemble_pred = (ensemble_pred_prob > 0.5).astype(int).flatten()
    
    # Ensemble metrics
    ensemble_acc = np.mean(ensemble_pred == y_test)
    ensemble_auc = roc_auc_score(y_test, ensemble_pred_prob)
    
    mean_individual_acc = np.mean(fold_individual_accs)
    improvement = ensemble_acc - mean_individual_acc
    
    # Store metrics
    ensemble_metrics['fold_no'].append(fold_no)
    ensemble_metrics['individual_accs'].append(fold_individual_accs)
    ensemble_metrics['ensemble_acc'].append(ensemble_acc)
    ensemble_metrics['ensemble_auc'].append(ensemble_auc)
    ensemble_metrics['individual_aucs'].append(fold_individual_aucs)
    ensemble_metrics['improvement'].append(improvement)
    
    print(f"\n{'='*60}")
    print(f"📊 FOLD {fold_no} ENSEMBLE RESULTS")
    print(f"{'='*60}")
    print(f"Individual Models:")
    for i, (acc, auc) in enumerate(zip(fold_individual_accs, fold_individual_aucs), 1):
        print(f"  Model {i}: Acc={acc:.4f}, AUC={auc:.4f}")
    print(f"\nMean Individual: Acc={mean_individual_acc:.4f}, AUC={np.mean(fold_individual_aucs):.4f}")
    print(f"Ensemble:        Acc={ensemble_acc:.4f}, AUC={ensemble_auc:.4f}")
    print(f"Improvement:     +{improvement:.4f} ({improvement*100:.2f}%)")
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, ensemble_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                annot_kws={'size': 14})
    plt.title(f"Fold {fold_no} Ensemble Confusion Matrix\nAcc: {ensemble_acc:.4f} | AUC: {ensemble_auc:.4f}", 
              fontsize=14)
    plt.xlabel("Predicted", fontsize=12)
    plt.ylabel("True", fontsize=12)
    plt.tight_layout()
    plt.savefig(f"results/ensemble/confusion_fold{fold_no}_ensemble.png", dpi=300)
    plt.close()
    
    # Classification Report
    print(f"\nEnsemble Classification Report:")
    print(classification_report(y_test, ensemble_pred, target_names=class_names, digits=4))


# ===================== FINAL SUMMARY =====================
print("\n" + "="*80)
print(" ENSEMBLE CROSS-VALIDATION SUMMARY")
print("="*80)

mean_individual_acc = np.mean([np.mean(accs) for accs in ensemble_metrics['individual_accs']])
mean_ensemble_acc = np.mean(ensemble_metrics['ensemble_acc'])
mean_ensemble_auc = np.mean(ensemble_metrics['ensemble_auc'])
mean_improvement = np.mean(ensemble_metrics['improvement'])

print(f"\n📊 Mean Individual Model Accuracy: {mean_individual_acc:.4f}")
print(f"📊 Mean Ensemble Accuracy:         {mean_ensemble_acc:.4f}")
print(f"📊 Mean Ensemble AUC:              {mean_ensemble_auc:.4f}")
print(f"🎯 Mean Improvement:               +{mean_improvement:.4f} ({mean_improvement*100:.2f}%)")

print(f"\n{'─'*60}")
print("Fold-by-Fold Results:")
print(f"{'─'*60}")
for i in range(len(ensemble_metrics['fold_no'])):
    fold = ensemble_metrics['fold_no'][i]
    individual_mean = np.mean(ensemble_metrics['individual_accs'][i])
    ensemble = ensemble_metrics['ensemble_acc'][i]
    improvement = ensemble_metrics['improvement'][i]
    
    print(f"Fold {fold}: Individual={individual_mean:.4f} → Ensemble={ensemble:.4f} "
          f"(+{improvement:.4f})")

# Visualization: Ensemble vs Individual
plt.figure(figsize=(12, 6))

# Plot 1: Accuracy comparison
plt.subplot(1, 2, 1)
folds = ensemble_metrics['fold_no']
individual_means = [np.mean(accs) for accs in ensemble_metrics['individual_accs']]
ensemble_accs = ensemble_metrics['ensemble_acc']

x = np.arange(len(folds))
width = 0.35

plt.bar(x - width/2, individual_means, width, label='Individual (mean)', alpha=0.8, color='skyblue')
plt.bar(x + width/2, ensemble_accs, width, label='Ensemble', alpha=0.8, color='coral')

plt.xlabel('Fold', fontsize=11)
plt.ylabel('Accuracy', fontsize=11)
plt.title('Individual vs Ensemble Accuracy', fontsize=13)
plt.xticks(x, folds)
plt.legend()
plt.grid(alpha=0.3, axis='y')

# Plot 2: Improvement per fold
plt.subplot(1, 2, 2)
improvements = [imp * 100 for imp in ensemble_metrics['improvement']]
colors = ['green' if imp > 0 else 'red' for imp in improvements]
plt.bar(folds, improvements, color=colors, alpha=0.7)
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
plt.xlabel('Fold', fontsize=11)
plt.ylabel('Improvement (%)', fontsize=11)
plt.title('Ensemble Improvement per Fold', fontsize=13)
plt.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig("results/ensemble/ensemble_summary.png", dpi=300)
plt.close()

print(f"\n💾 All results saved in 'results/ensemble/' directory")
print("\n✅ Ensemble training complete!")

Original classes: ['ICTAL' 'INTERICTAL' 'NORMAL']
Binary classes: 0 (NORMAL) vs 1 (INTERICTAL + ICTAL)
Binary labels distribution: (array([0, 1]), array([4400, 6600], dtype=int64))
Dataset shape: (11000, 347, 1)
Class weights: {0: 1.25, 1: 0.8333333333333334}

 ENSEMBLE TRAINING: 5 MODELS

🎲 Base random state for fold splitting: 8006

 FOLD 1 - TRAINING 5 MODELS

Train: 7480, Val: 1320, Test: 2200

────────────────────────────────────────────────────────────
🔸 Training Model 1/5 (seed=42)
────────────────────────────────────────────────────────────


d:\anaconda\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


d:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  ✓ Model 1 - Acc: 0.9691, AUC: 0.9956

────────────────────────────────────────────────────────────
🔸 Training Model 2/5 (seed=123)
────────────────────────────────────────────────────────────


d:\anaconda\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
d:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

  ✓ Model 4 - Acc: 0.9700, AUC: 0.9966

────────────────────────────────────────────────────────────
🔸 Training Model 5/5 (seed=1024)
────────────────────────────────────────────────────────────


d:\anaconda\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
d:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  ✓ Model 5 - Acc: 0.9705, AUC: 0.9962

📊 FOLD 1 ENSEMBLE RESULTS
Individual Models:
  Model 1: Acc=0.9691, AUC=0.9956
  Model 2: Acc=0.9736, AUC=0.9968
  Model 3: Acc=0.9718, AUC=0.9965
  Model 4: Acc=0.9700, AUC=0.9966
  Model 5: Acc=0.9705, AUC=0.9962

Mean Individual: Acc=0.9710, AUC=0.9964
Ensemble:        Acc=0.9755, AUC=0.9966
Improvement:     +0.0045 (0.45%)

Ensemble Classification Report:
                        precision    recall  f1-score   support

                NORMAL     0.9672    0.9716    0.9694       880
ALL (INTERICTAL+ICTAL)     0.9810    0.9780    0.9795      1320

              accuracy                         0.9755      2200
             macro avg     0.9741    0.9748    0.9745      2200
          weighted avg     0.9755    0.9755    0.9755      2200


 FOLD 2 - TRAINING 5 MODELS

Train: 7480, Val: 1320, Test: 2200

────────────────────────────────────────────────────────────
🔸 Training Model 1/5 (seed=42)
─────────────────────────────────────────────────────

d:\anaconda\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
d:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 